<a href="https://colab.research.google.com/github/JonasZimmer1994/deeplearning_wavenet/blob/master/wavenet_pytorch_abgabe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hier werden zunächst die wichtigen Scripte von GitHub geklont


In [0]:
!git clone https://github.com/vincentherrmann/pytorch-wavenet.git

Danach navigiert man in den Pfad der Datei die gerade heruntergeladen wurde

In [0]:
%cd /content/pytorch-wavenet/

Es wird die Version 0.3.0 von pytorch benötigt. Deshalb muss sie heruntergeladen und installiert werden.

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

von Tensorflow wird die Version 1.12.0 benötigt. Diese wird hier heruntergeladen und installiert.

In [0]:
!pip install tensorflow==1.12.0
import tensorflow as tf
print(tf.__version__)

Die nächsten vier Code-Blöcke sind zur Aktivierung des Tensorboards.

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = 'logs/chaconne_model'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Zunächst werden alle nötigen Pakete importiert

In [0]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

anschließend wird, wenn eine GPU vorhanden ist, der Datentyp und das Label auf den Cuda-Datentyp initialisiert

In [0]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else: 
    print('gpu not available')

Hierin wird das eigentliche Neuronale Netz erstellt.

In [0]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16,
                     dtype=dtype, 
                     bias=True)
#model = load_latest_model_from('snapshots', use_cuda=use_cuda)

#model = model.cuda()

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

In diesem Codeblock werden die Trainingsdaten initialisiert. Diese müssen sich in dem Ordner train_samples/bach_chaconne befinden. Der Code sorgt dafür, dass diese dann in eine dataset.npz Datei umgewandelt werden.

In [0]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

Sofern der Tensorboardlogger verwendet wird, wird diese Funktion zum loggen verwendet. Zudem sollte Sie Zwischenergebnisse Ausgeben.

In [0]:
def generate_and_log_samples(step):
    sample_length=32000


    gen_model = load_latest_model_from('snapshots', use_cuda=False)
   
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")

Es kann entweder der TensorboardLogger verwendet werden, wodurch man sein Training auf Tensorboard nachverfolgen kann, oder ein einfacher Logger, der die logging-Daten auf der Konsole ausgibt.

In [0]:
logger = TensorboardLogger(log_interval=200,
                           validation_interval=400,
                           generate_interval=1000,
                           generate_function=generate_and_log_samples,
                           log_dir="logs/chaconne_model")

#logger = Logger(log_interval=200,
#                validation_interval=400,
#                generate_interval=1000)

In dem folgenden Codeblock wird das Wavenet Model trainiert. Hierzu wird ein trainer erstellt, dem die batch_size und die Epochenzahl beim Aufruf mitgegeben wird.

In [0]:
trainer = WavenetTrainer(model=model.cuda(),
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)


print('start training...')
trainer.train(batch_size=6,
              epochs=10)

In diesem Codeblock, wird anhand des trainierten Models eine Ausgabe erzeugt. Die Länge der erzeugten Ausgabe kann anhand der Variable num_samples variiert werden. 16000 Samples ergeben eine Sekunde Ausgabe. Die rate ist für die Abspielgeschwindigkeit der Ausgabedatei verantwortlich

In [0]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

model.cpu()
generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)


import IPython.display as ipd

ipd.Audio(generated, rate=16000)